In [1]:
%matplotlib inline
'''
Stripping, 3 stage set up, demonstrate final stage vapor disappears
'''
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# import pickle
import dill
from copy import deepcopy

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, \
                                    plot_distribution, plot_product_distribution, check_product_spec
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, check_iteration, tray_translator
from utility.model_utility import which_MPCC, select_MPCC, augmented_objective, add_solver, disable_restoration
from utility.time_utility import create_filename_time, log_now, log_end

2018-09-26 21:22:59 - Start Program


In [2]:
model = pe.ConcreteModel(name='reactive_distillation')

In [3]:
mpcc_type = 'pf'
tray_number = 3

In [4]:
model.TRAY = pe.RangeSet(1,tray_number)

In [5]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)
# change reactive flash MPCC type based on model input
for j in model.reactive:
    select_MPCC(model.reactive[j],mpcc_type)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].H_F
| reactive[1].T
| reactive[1].T_F
| reactive[1].P
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].r_total_comp
| reactive[1].PR_L
| reactive[1].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_b

In [6]:
'''
connect reactive stages
'''

def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

In [7]:
for j in model.reactive:
    model.reactive[j].x_.fix(0)
    model.reactive[j].y_.fix(0)
    model.reactive[j].L['in'].fix(0)
    model.reactive[j].V['in'].fix(0)
    model.reactive[j].L['R'].fix(0)
    model.reactive[j].V['R'].fix(0)
    model.reactive[j].H_L_.fix(0)
    model.reactive[j].H_V_.fix(0)

# operating parameters
for j in model.reactive:
    model.reactive[j].cat.fix(3000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)

    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0/2)
    model.reactive[j].z['C30H62'].fix(0)

    model.reactive[j].PR_L.fix(0)
    model.reactive[j].PR_V.fix(0)

    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(200+273.15)
    model.reactive[j].T.setlb(200+273.15)

In [8]:
for j in model.reactive:
    if j != model.TRAY.first():
        for i in m.COMP_TOTAL:
            model.reactive[j].x_['in',i].unfix()
        model.reactive[j].H_L_['in'].unfix()
        model.reactive[j].L['in'].unfix()
    if j != model.TRAY.last():
        for i in m.COMP_TOTAL:
            model.reactive[j].y_['in',i].unfix()
        model.reactive[j].V['in'].unfix()
        model.reactive[j].H_V_['in'].unfix()

In [9]:
opt = add_solver(pe, max_iter = 5000)
disable_restoration(mode = 'enable')

In [10]:
model.obj = augmented_objective(pe,model,expr = sum(model.reactive[j].T for j in model.reactive),sense=pe.maximize)
add_dual(pe,model)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[1].T + reactive[2].T + reactive[3].T - reactive[1].MPCC_P_pf.pf - reactive[2].MPCC_P_pf.pf - reactive[3].MPCC_P_pf.pf
------------------------------------------------------------------------------------------------------------
Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [11]:
model.reactive[1].cat.fix(3000)
model.reactive[1].F.fix(1)
model.reactive[1].T.setub(200+273.15)

model.reactive[2].cat.fix(3000)
model.reactive[2].F.fix(1)
model.reactive[2].T.setub(200+273.15)

model.reactive[3].cat.fix(3000)
model.reactive[3].F.fix(1)
model.reactive[3].T.setub(200+273.15)

* DOF, 3 stage temperatures

In [12]:
check_DOF(pe,model)

Active Equality Constraints:	 3888
Active Inequality Constraints:	 3
Active Variables:		 4578
Fixed Variables:		 678
DOF:				 12


In [13]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.10: print_user_options=yes
tol=1e-08
option_file_name=./ipopt.opt
max_iter=5000

Using option file "./ipopt.opt".


List of user-set options:

                                    Name   Value                used
                           compl_inf_tol = 1                     yes
                         constr_viol_tol = 1                     yes
                            dual_inf_tol = 1000                  yes
                           linear_solver = ma97                  yes
                              ma86_small = 1e-30                  no
                                  ma86_u = 0.0001                 no
                              ma97_order = best                  yes
                              ma97_small = 1e-30                 yes
                            ma97_switch1 = at_start              yes
                            ma97_switch2 = od_hd_reuse           yes
                                  ma97_u = 1e-06                 yes
                  

  68 -1.4166335e+03 3.38e+01 3.79e+08  -1.0 4.75e+02    -  5.14e-02 7.01e-05h  1
  69r-1.4166335e+03 3.38e+01 1.00e+03   1.5 0.00e+00    -  0.00e+00 2.95e-07R  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70r-1.4166062e+03 2.04e+01 2.02e+05   1.5 3.37e+04    -  1.86e-02 3.95e-04f  1
  71 -1.4166063e+03 2.04e+01 2.74e+04  -1.0 4.74e+02    -  6.59e-01 9.54e-05f  1
  72 -1.4166136e+03 2.03e+01 8.58e+05  -1.0 4.74e+02    -  9.18e-01 5.37e-03h  1
  73 -1.4166136e+03 2.03e+01 4.01e+08  -1.0 4.72e+02    -  2.56e-02 5.47e-05h  1
  74r-1.4166136e+03 2.03e+01 1.00e+03   1.3 0.00e+00    -  0.00e+00 2.78e-07R  4
  75r-1.4165858e+03 9.90e+00 1.67e+05   1.3 2.03e+04    -  2.58e-02 4.91e-04f  1
  76 -1.4165859e+03 9.90e+00 2.22e+04  -1.0 4.70e+02    -  6.59e-01 9.91e-05h  1
  77 -1.4165917e+03 9.86e+00 1.51e+06  -1.0 4.70e+02    -  9.19e-01 4.09e-03h  1
  78 -1.4165912e+03 9.86e+00 3.97e+08  -1.0 4.69e+02    -  1.09e-02 4.14e-05h  1
  79r-1.4165912e+03 9.86e+00

Number of inequality constraint evaluations          = 359
Number of equality constraint Jacobian evaluations   = 180
Number of inequality constraint Jacobian evaluations = 180
Number of Lagrangian Hessian evaluations             = 162
Total CPU secs in IPOPT (w/o function evaluations)   =      2.059
Total CPU secs in NLP function evaluations           =      0.188

EXIT: Optimal Solution Found.


In [14]:
beautify_reactive(pe,model)

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
React[1]     200.0  -36.2       0.212  0.378  1.000  3000.       1.7230  0.0000  0.0067  0.0000       20.000
React[2]     200.0  -36.2       0.212  0.477  1.000  3000.       1.1487  0.0000  0.0135  0.0000       20.000
React[3]     200.0  -36.2       0.212  0.645  1.000  3000.       0.5743  0.0000  0.0202  0.0000       20.000


In [15]:
model.reactive[1].cat.fix(30000)
model.reactive[1].F.fix(10)
model.reactive[1].T.setub(220+273.15)

model.reactive[2].cat.fix(0)
model.reactive[2].F.fix(0)
model.reactive[2].T.setub(220+273.15)

model.reactive[3].cat.fix(0)
model.reactive[3].F.fix(0)
model.reactive[3].T.setub(350+273.15)

In [16]:
disable_restoration(mode = 'disable')
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.10: print_user_options=yes
tol=1e-08
option_file_name=./ipopt.opt
max_iter=5000

Using option file "./ipopt.opt".


List of user-set options:

                                    Name   Value                used
                           compl_inf_tol = 1                     yes
                         constr_viol_tol = 1                     yes
                            dual_inf_tol = 1000                  yes
                           linear_solver = ma97                  yes
                              ma86_small = 1e-30                  no
                                  ma86_u = 0.0001                 no
                              ma97_order = best                  yes
                              ma97_small = 1e-30                 yes
                            ma97_switch1 = at_start              yes
                            ma97_switch2 = od_hd_reuse           yes
                                  ma97_u = 1e-06                 yes
                  

  58 -1.4216660e+03 6.55e+01 3.77e+08  -1.0 1.27e+03    -  5.89e-01 3.79e-01f  1
  59 -1.4386034e+03 5.19e+01 7.26e+08  -1.0 3.85e+03    -  2.79e-01 2.09e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60 -1.4400602e+03 5.18e+01 2.29e+10  -1.0 8.27e+04    -  1.63e-02 2.09e-03f  1
  61 -1.4399919e+03 5.18e+01 1.09e+11  -1.0 7.69e+04    -  2.10e-02 1.36e-04h  8
  62 -1.4399223e+03 5.18e+01 2.57e+11  -1.0 8.63e+04    -  2.61e-02 1.21e-04h  8
  63 -1.4398513e+03 5.18e+01 5.22e+11  -1.0 9.85e+04    -  3.64e-02 1.06e-04h  8
  64 -1.4397788e+03 5.18e+01 7.19e+11  -1.0 1.15e+05    -  2.88e-02 9.09e-05h  8
  65 -1.4397416e+03 5.18e+01 6.44e+11  -1.0 1.38e+05    -  2.61e-02 3.78e-05h  9
  66 -1.4397033e+03 5.18e+01 1.64e+11  -1.0 1.53e+05    -  9.10e-03 3.40e-05h  9
  67r-1.4397033e+03 5.18e+01 1.00e+03   1.2 0.00e+00    -  0.00e+00 4.79e-07R 15
  68r-1.4396556e+03 5.17e+01 5.83e+04   1.2 1.64e+04    -  1.31e-03 2.35e-04f  1
  69 -1.4396558e+03 5.17e+01

In [17]:
beautify_reactive(pe,model)

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
React[1]     220.0  -432.       2.660  0.815  10.00  30000       4.7795  0.0000  0.0642  0.0000       20.000
NON--[2]     220.0  -0.22       0.000  0.000  0.000  0.000       0.0032  0.0000  0.0678  0.0000       20.000
NON--[3]     350.0  6.744       0.000  0.000  0.000  0.000       0.0068  0.0000  0.0610  0.0000       20.000


In [18]:
feed_range = [0,0.0001,0.001,0.01,0.1,1]
for feed in feed_range:
    model.reactive[1].F.fix(10-feed)
    model.reactive[2].F.fix(feed)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    beautify_reactive(pe,model)

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
React[1]     220.0  -432.       2.660  0.815  10.00  30000       4.7795  0.0000  0.0642  0.0000       20.000
NON--[2]     220.0  -0.22       0.000  0.000  0.000  0.000       0.0032  0.0000  0.0678  0.0000       20.000
NON--[3]     350.0  6.744       0.000  0.000  0.000  0.000       0.0068  0.0000  0.0610  0.0000       20.000
stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
React[1]     220.0  -432.       2.660  0.815  9.999  30000       4.7795  0.0000  0.0642  0.0000       20.000
NON--[2]     220.0  -0.21       0.000  0.000  0.000  0.000       0.0033  0.0000  0.0676  0.0000       20.000
NON--[3]     350.0  6.732       0.000  0.000  0.000  0.000       0.0066  0.0000  0.0609  0.0000       20.000
stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
React[1]     220.0 